In [1]:
# !! You have to restart session after executing this cell !!
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

     |████████████████████████████████| 11.3 MB 32.4 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.


     |████████████████████████████████| 109 kB 17.1 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


In [1]:
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
# Write a path for saving models
root = '/gdrive/My Drive/CS492_Project/LSTM_Models'

In [4]:
                                                              
#importing packages
import pandas as pd
import numpy as np
import datetime as dt
import pandas_datareader as pdr
import seaborn as sns
import matplotlib.pyplot as plt
import bs4 as bs
import requests
from IPython.display import clear_output
from scipy.stats import mstats
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import RandomizedSearchCV, validation_curve, TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier
import pickle
import os
sns.set()

#Obtain list of S&P100 companies from wikipedia
resp = requests.get("https://en.wikipedia.org/wiki/S%26P_100")
convert_soup = bs.BeautifulSoup(resp.text, 'lxml')
table = convert_soup.find('table',{'class':'wikitable sortable'})

tickers = []

for rows in table.findAll('tr')[1:]:
    ticker = rows.findAll('td')[0].text.strip()
    tickers.append(ticker)

all_data = pd.DataFrame()
test_data = pd.DataFrame()
no_data = []

#Extract data from Yahoo Finance
for i in tickers:
    try:
        print(i)
        test_data = pdr.get_data_yahoo(i, start = dt.datetime(2015,1,1), end = dt.datetime(2019,12,31))
        test_data['symbol'] = i
        all_data = all_data.append(test_data)
        clear_output(wait = True)
    except:
        no_data.append(i)

    clear_output(wait = True)

XOM


In [5]:
# Simple Moving Average
all_data['SMA_5'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.rolling(window = 5).mean())
all_data['SMA_15'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.rolling(window = 15).mean())
all_data['SMA_ratio'] = all_data['SMA_15'] / all_data['SMA_5']

In [6]:
# Simple Moving Average of Volume
all_data['SMA5_Volume'] = all_data.groupby('symbol')['Volume'].transform(lambda x: x.rolling(window = 5).mean())
all_data['SMA15_Volume'] = all_data.groupby('symbol')['Volume'].transform(lambda x: x.rolling(window = 15).mean())
all_data['SMA_Volume_Ratio'] = all_data['SMA5_Volume']/all_data['SMA15_Volume']

In [7]:
def Wilder(data, periods):
    start = np.where(~np.isnan(data))[0][0] #Check if nans present in beginning
    Wilder = np.array([np.nan]*len(data))
    Wilder[start+periods-1] = data[start:(start+periods)].mean() #Simple Moving Average
    for i in range(start+periods,len(data)):
        Wilder[i] = (Wilder[i-1]*(periods-1) + data[i])/periods #Wilder Smoothing
    return(Wilder)

In [8]:
# Average True Return

all_data['prev_close'] = all_data.groupby('symbol')['Close'].shift(1)
all_data['TR'] = np.maximum((all_data['High'] - all_data['Low']), 
                     np.maximum(abs(all_data['High'] - all_data['prev_close']), 
                     abs(all_data['prev_close'] - all_data['Low'])))
for i in all_data['symbol'].unique():
    TR_data = all_data[all_data.symbol == i].copy()
    all_data.loc[all_data.symbol==i,'ATR_5'] = Wilder(TR_data['TR'], 5)
    all_data.loc[all_data.symbol==i,'ATR_15'] = Wilder(TR_data['TR'], 15)

all_data['ATR_Ratio'] = all_data['ATR_5'] / all_data['ATR_15']

In [9]:
# Average Directional Movement

all_data['prev_high'] = all_data.groupby('symbol')['High'].shift(1)
all_data['prev_low'] = all_data.groupby('symbol')['Low'].shift(1)

all_data['+DM'] = np.where(~np.isnan(all_data.prev_high),
                           np.where((all_data['High'] > all_data['prev_high']) & 
         (((all_data['High'] - all_data['prev_high']) > (all_data['prev_low'] - all_data['Low']))), 
                                                                  all_data['High'] - all_data['prev_high'], 
                                                                  0),np.nan)

all_data['-DM'] = np.where(~np.isnan(all_data.prev_low),
                           np.where((all_data['prev_low'] > all_data['Low']) & 
         (((all_data['prev_low'] - all_data['Low']) > (all_data['High'] - all_data['prev_high']))), 
                                    all_data['prev_low'] - all_data['Low'], 
                                    0),np.nan)

for i in all_data['symbol'].unique():
    ADX_data = all_data[all_data.symbol == i].copy()
    all_data.loc[all_data.symbol==i,'+DM_5'] = Wilder(ADX_data['+DM'], 5)
    all_data.loc[all_data.symbol==i,'-DM_5'] = Wilder(ADX_data['-DM'], 5)
    all_data.loc[all_data.symbol==i,'+DM_15'] = Wilder(ADX_data['+DM'], 15)
    all_data.loc[all_data.symbol==i,'-DM_15'] = Wilder(ADX_data['-DM'], 15)

all_data['+DI_5'] = (all_data['+DM_5']/all_data['ATR_5'])*100
all_data['-DI_5'] = (all_data['-DM_5']/all_data['ATR_5'])*100
all_data['+DI_15'] = (all_data['+DM_15']/all_data['ATR_15'])*100
all_data['-DI_15'] = (all_data['-DM_15']/all_data['ATR_15'])*100

all_data['DX_5'] = (np.round(abs(all_data['+DI_5'] - all_data['-DI_5'])/(all_data['+DI_5'] + all_data['-DI_5']) * 100))

all_data['DX_15'] = (np.round(abs(all_data['+DI_15'] - all_data['-DI_15'])/(all_data['+DI_15'] + all_data['-DI_15']) * 100))

for i in all_data['symbol'].unique():
    ADX_data = all_data[all_data.symbol == i].copy()
    all_data.loc[all_data.symbol==i,'ADX_5'] = Wilder(ADX_data['DX_5'], 5)
    all_data.loc[all_data.symbol==i,'ADX_15'] = Wilder(ADX_data['DX_15'], 15)

In [10]:
# Stochastic Oscillator

all_data['Lowest_5D'] = all_data.groupby('symbol')['Low'].transform(lambda x: x.rolling(window = 5).min())
all_data['High_5D'] = all_data.groupby('symbol')['High'].transform(lambda x: x.rolling(window = 5).max())
all_data['Lowest_15D'] = all_data.groupby('symbol')['Low'].transform(lambda x: x.rolling(window = 15).min())
all_data['High_15D'] = all_data.groupby('symbol')['High'].transform(lambda x: x.rolling(window = 15).max())

all_data['Stochastic_5'] = ((all_data['Close'] - all_data['Lowest_5D'])/(all_data['High_5D'] - all_data['Lowest_5D']))*100
all_data['Stochastic_15'] = ((all_data['Close'] - all_data['Lowest_15D'])/(all_data['High_15D'] - all_data['Lowest_15D']))*100

all_data['Stochastic_%D_5'] = all_data['Stochastic_5'].rolling(window = 5).mean()
all_data['Stochastic_%D_15'] = all_data['Stochastic_5'].rolling(window = 15).mean()

all_data['Stochastic_Ratio'] = all_data['Stochastic_%D_5']/all_data['Stochastic_%D_15']

In [11]:
# Relative Strength Index

all_data['Diff'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.diff())
all_data['Up'] = all_data['Diff']
all_data.loc[(all_data['Up']<0), 'Up'] = 0

all_data['Down'] = all_data['Diff']
all_data.loc[(all_data['Down']>0), 'Down'] = 0 
all_data['Down'] = abs(all_data['Down'])

all_data['avg_5up'] = all_data.groupby('symbol')['Up'].transform(lambda x: x.rolling(window=5).mean())
all_data['avg_5down'] = all_data.groupby('symbol')['Down'].transform(lambda x: x.rolling(window=5).mean())

all_data['avg_15up'] = all_data.groupby('symbol')['Up'].transform(lambda x: x.rolling(window=15).mean())
all_data['avg_15down'] = all_data.groupby('symbol')['Down'].transform(lambda x: x.rolling(window=15).mean())

all_data['RS_5'] = all_data['avg_5up'] / all_data['avg_5down']
all_data['RS_15'] = all_data['avg_15up'] / all_data['avg_15down']

all_data['RSI_5'] = 100 - (100/(1+all_data['RS_5']))
all_data['RSI_15'] = 100 - (100/(1+all_data['RS_15']))

all_data['RSI_ratio'] = all_data['RSI_5']/all_data['RSI_15']

In [12]:
# Moving Average Convergence Divergence
all_data['5Ewm'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.ewm(span=5, adjust=False).mean())
all_data['15Ewm'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.ewm(span=15, adjust=False).mean())
all_data['MACD'] = all_data['15Ewm'] - all_data['5Ewm']

In [13]:
all_data['15MA'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.rolling(window=15).mean())
all_data['SD'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.rolling(window=15).std())
all_data['upperband'] = all_data['15MA'] + 2*all_data['SD']
all_data['lowerband'] = all_data['15MA'] - 2*all_data['SD']

In [14]:
all_data['RC'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.pct_change(periods = 15)) 

In [15]:
# Target Direction and Moving Percent
all_data['Close_Shifted'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.shift(-6))
all_data['Close_Yester'] = all_data.groupby('symbol')['Close'].transform(lambda x: x.shift(1))
all_data['Moving_Percent'] = ((all_data['Close_Yester'] - all_data['Close'])/(all_data['Close_Yester']) * 100)
all_data['Target'] = ((all_data['Close_Shifted'] - all_data['Open'])/(all_data['Open']) * 100)
all_data['Target_Direction'] = np.where(all_data['Target']>0,1,0)
all_data = all_data.dropna().copy()

In [16]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

def train_lstm (Train_X, Train_Y, activation, config_num, window_len):
  model= Sequential()

  model.add(LSTM(units = 50,return_sequences=True,activation=activation,input_shape=(Train_X.shape[1], Train_X.shape[2])))
  model.add(Dropout(0.2))

  model.add(LSTM(units=50, activation=activation,return_sequences=True))
  model.add(Dropout(0.2))

  model.add(LSTM(units=50, activation=activation, return_sequences=True))
  model.add(Dropout(0.2))

  model.add(LSTM(units=50, activation=activation))
  model.add(Dropout(0.2))
  model.add(Dense(units=1))

  model.compile(optimizer='adam',loss='mse')

  # model.summary()
  model.fit(Train_X, Train_Y ,batch_size=32, epochs=5)

  file_loc = f'{root}/config_{config_num}_{window_len}' 
  pickle.dump(model, open(file_loc,'wb'))

In [17]:
def dataset_generator (dataset, x_label, y_label, targets, window_len):
  for i in range (window_len, len(dataset) - window_len):
    if not targets == None:
      x = dataset.loc[i-window_len:i-1, targets]
    else:
       x = dataset.iloc[i]
    x_label.append(x)

    if not y_label == None:
      y = dataset.loc[i, ['Target_Direction']]
      y_label.append(y)

In [ ]:
total_target_var = ['SMA_ratio','ATR_5','ATR_15','ATR_Ratio',
                       'ADX_5','ADX_15','SMA_Volume_Ratio','Stochastic_5','Stochastic_15','Stochastic_Ratio',
                      'RSI_5','RSI_15','RSI_ratio','MACD']

In [18]:
vc_dict = {
    "Baseline": [], 
    "SMA_Price": [0], 
    "ATR": [1, 2, 3], 
    "ADX": [4, 5], 
    "SMA_Volume": [6], 
    "Stochastic": [7, 8, 9], 
    "RSI": [10, 11, 12], 
    "MACD": [13],
    "All_feature": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]}

In [19]:
"""
  Train single models and save it in root

  config_num:
  0 -> baseline
  1 -> SMA_Price 
  2 -> ATR 
  3 -> ADX 
  4 -> SMA_Volume 
  5 -> Stochastic 
  6 -> RSI 
  7 -> MACD
  8 -> All feature
"""

from sklearn.preprocessing import MinMaxScaler
import copy

window_length = 8
activation = 'relu'

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

for config_num, vc in enumerate(vc_dict.values()):
  Target_variables = []
  train_data = all_data.copy()
  for ti in vc:
    Target_variables.append(total_target_var[ti])
  Target_variables.append('Moving_Percent')
  for variable in Target_variables:
    train_data.loc[:,variable] = mstats.winsorize(train_data.loc[:,variable], limits = [0.1,0.1])
  for variable in Target_variables:
    train_data[[variable]] = min_max_scaler.fit_transform(train_data[[variable]])
  train_data.index = pd.to_datetime(train_data.index)
  Train_X = []
  Train_Y = []
  for i in tickers:
    train_data_sym = train_data[test_data['symbol'] == i]
    dataset_generator (train_data_sym.reset_index(), Train_X, Train_Y, Target_variables, window_length)
  Train_X = np.array(Train_X).astype(float)
  Train_Y = np.array(Train_Y).astype(float)
  train_lstm (Train_X, Train_Y, activation, config_num, window_length)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Epoch 1/5
3781/3781 [==============================] - 81s 20ms/step - loss: 0.2513
Epoch 2/5
3781/3781 [==============================] - 75s 20ms/step - loss: 0.2481
Epoch 3/5
3781/3781 [==============================] - 75s 20ms/step - loss: 0.2476
Epoch 4/5
3781/3781 [==============================] - 75s 20ms/step - loss: 0.2473
Epoch 5/5
3781/3781 [==============================] - 75s 20ms/step - loss: 0.2471
INFO:tensorflow:Assets written to: ram://a6a25827-d868-4b54-9259-063207825245/assets


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Epoch 1/5
3781/3781 [==============================] - 80s 20ms/step - loss: 0.2514
Epoch 2/5
3781/3781 [==============================] - 76s 20ms/step - loss: 0.2482
Epoch 3/5
3781/3781 [==============================] - 75s 20ms/step - loss: 0.2476
Epoch 4/5
3781/3781 [==============================] - 75s 20ms/step - loss: 0.2471
Epoch 5/5
3781/3781 [==============================] - 75s 20ms/step - loss: 0.2468
INFO:tensorflow:Assets written to: ram://f2183741-73bd-437d-8634-c981397e87c2/assets


INFO:tensorflow:Assets written to: ram://f2183741-73bd-437d-8634-c981397e87c2/assets
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


...

INFO:tensorflow:Assets written to: ram://c67e531e-a2b9-4506-8183-575540c86756/assets
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Epoch 1/5
3781/3781 [==============================] - 80s 20ms/step - loss: 0.2509
Epoch 2/5
3781/3781 [==============================] - 75s 20ms/step - loss: 0.2482
Epoch 3/5
3781/3781 [==============================] - 76s 20ms/step - loss: 0.2475
Epoch 4/5
3781/3781 [==============================] - 75s 20ms/step - loss: 0.2470
Epoch 5/5
3781/3781 [==============================] - 75s 20ms/step - loss: 0.2466
INFO:tensorflow:Assets written to: ram://4189c1ac-cefc-48f4-9225-381e5e8f0214/assets


INFO:tensorflow:Assets written to: ram://4189c1ac-cefc-48f4-9225-381e5e8f0214/assets
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Epoch 1/5
3781/3781 [==============================] - 80s 20ms/step - loss: 0.2510
Epoch 2/5
3781/3781 [==============================] - 76s 20ms/step - loss: 0.2481
Epoch 3/5
3781/3781 [==============================] - 76s 20ms/step - loss: 0.2474
Epoch 4/5
3781/3781 [==============================] - 76s 20ms/step - loss: 0.2468
Epoch 5/5
3781/3781 [==============================] - 76s 20ms/step - loss: 0.2463


KeyboardInterrupt: ignored